In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import pymc
import scipy.stats as stats
import pymc as pm
import arviz as az

from sklearn.preprocessing import StandardScaler

sns.set_style("whitegrid")

In [3]:
df1 = pd.read_csv(r"data_01.csv")
df2 = pd.read_csv(r"data_01.csv")
df3 = pd.read_csv(r"data_01.csv")
df4 = pd.read_csv(r"data_01.csv")
df5 = pd.read_csv(r"data_01.csv")
df6 = pd.read_csv(r"data_01.csv")

In [4]:
dfs = [df1,df2,df3,df4,df5,df6]

In [5]:
all_data['x_s'] = all_data.x - all_data.x.mean()

NameError: name 'all_data' is not defined

In [ ]:

all_data = pd.concat(datasets, keys=range(len(dfs)))


all_data['dataset_idx'] = np.repeat(range(len(dfs)), [len(df) for df in dfs])

M = len(dfs)
N = len(all_data)

with pm.Model() as unpooled_model:
    α_tmp = pm.Normal('α_tmp', mu=0, sigma=10, shape=M)
    β = pm.Normal('β', mu=0, sigma=10, shape=M)
    ε = pm.HalfCauchy('ε', 5)
    ν = pm.Exponential('ν', 1/30)

    y_pred = pm.StudentT('y_pred', mu=α_tmp[all_data['dataset_idx']] + β[all_data['dataset_idx']] * all_data['x'].values,
                         sigma=ε, nu=ν, observed=all_data['y'].values)

    α = pm.Deterministic('α', α_tmp - β * all_data.groupby('dataset_idx').mean()['x'].values)

    trace_up = pm.sample(2000, tune=1000, target_accept=0.8)

az.plot_forest(trace_up, var_names=['α', 'β'], combined=True);

In [ ]:
with pm.Model() as hierarchical_model:
    α_μ_tmp = pm.Normal('α_μ_tmp', mu=0, sigma=5)
    α_σ_tmp = pm.HalfNormal('α_σ_tmp', sigma=3)
    β_μ = pm.Normal('β_μ', mu=0, sigma=2)
    β_σ = pm.HalfNormal('β_σ', sigma=2)

    α_tmp = pm.Normal('α_tmp', mu=α_μ_tmp, sigma=α_σ_tmp, shape=len(dfs))
    β = pm.Normal('β', mu=β_μ, sigma=β_σ, shape=len(dfs))
    ε = pm.HalfCauchy('ε', 2)
    ν = pm.Normal('ν', 1/12)

    y_pred = pm.StudentT('y_pred', mu=α_tmp[all_data['dataset_idx']] + β[all_data['dataset_idx']] * all_data['x'].values,
                         sigma=ε, nu=ν, observed=all_data['y'].values)

    α = pm.Deterministic('α', α_tmp - β * all_data.groupby('dataset_idx').mean()['x'].values)
    α_μ = pm.Deterministic('α_μ', α_μ_tmp - β_μ * all_data.groupby('dataset_idx').mean()['x'].values)
    α_σ = pm.Deterministic('α_sd', α_σ_tmp - β_μ * all_data.groupby('dataset_idx').mean()['x'].values)

    trace_hm = pm.sample(1000, target_accept=0.95)

# Отобразите результаты
az.plot_forest(trace_hm, var_names=['α', 'β'], combined=True);

In [ ]:
_, ax = plt.subplots(2, 4, figsize=(15, 10), sharex=True, sharey=True, constrained_layout=True)

trace_hm1 = trace_hm.posterior.stack(samples={"chain", "draw"})

ax = np.ravel(ax)
j, k = 0, 6
x_range = np.linspace(all_data['x'].min() - 1, all_data['x'].max() + 1, 10)

for i in range(6):
    ax[i].scatter(all_data[all_data['dataset_idx'] == i]['x'], all_data[all_data['dataset_idx'] == i]['y'])
    ax[i].set_xlabel(f'x_{i}')
    ax[i].set_ylabel(f'y_{i}', labelpad=17, rotation=0)

    # Получите средние значения параметров из трассировки
    alpha_hm1 = trace_hm1['α'].sel({'α_dim_0': i}).mean().item()
    beta_hm1 = trace_hm1['β'].sel({'β_dim_0': i}).mean().item()
    alpha_hm1 = alpha_hm1+9
    ax[i].plot(x_range, alpha_hm1 + beta_hm1 * x_range, c='r', label=f'y = {alpha_hm1:.2f} + {beta_hm1:.2f} * x' + ' - иерахич. модель')

    ax[i].legend()
    plt.xlim(all_data['x'].min() - 1, all_data['x'].max() + 1)
    plt.ylim(all_data['y'].min() - 1, all_data['y'].max() + 1)

In [ ]:
3